# [Классификация текста с предварительно обработанным текстом: обзоры фильмов](https://www.tensorflow.org/tutorials/keras/text_classification)



Мы будем использовать [набор данных IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) содержащий тексты 50 000 обзоров фильмов из [базы данных фильмов в Интернете](https://www.imdb.com/). Они разделены на 25 000 обзоров для обучения и 25 000 обзоров для проверки моделей. Обучающая и тестовая выборка сбалансированы , т.е. содержат одинаковое количество *позитивных* и *негативных обзоров*.

В качестве углубленного изучения полезно изучить статью "Word2vec в картинках" на [habr.ru](https://habr.com/ru/post/446530/).


In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## Загружаем датасет



Датасет IMDB доступен в [датасетах TensorFlow](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets). Следующий код скачивает датасет IMDB на ваш компьютер (или в среду выполнения Colab):

In [ ]:
(train_data, test_data), info = tfds.load(
    # Воспользуемся версией, предварительно закодированная с ~ 8K словарем.
    'imdb_reviews/subwords8k', 
    # Возращает тренировочный/тестовый наборы данных как кортеж.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Возрашать (примеры, лейблы) парами из набора данных (вместо словаря).
    as_supervised=True,
    # Также попросим вернуть структуру "информация".
    with_info=True)

## Кодировщик текста


Набор данных info включает кодировщик текста (см. [tfds.features.text.SubwordTextEncoder](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/SubwordTextEncoder)).

In [ ]:
encoder = info.features['text'].encoder

In [ ]:
print ('Vocabulary size: {}'.format(encoder.vocab_size))

Этот текстовый кодировщик обратимо закодирует любую строку:

In [ ]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print ('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('The original string: "{}"'.format(original_string))

assert original_string == sample_string

Кодировщик кодирует строку, разбивая ее на подслова или символы, если слово отсутствует в его словаре. Таким образом, чем больше строка напоминает набор данных, тем короче будет закодированное представление.

In [ ]:
for ts in encoded_string:
  print ('{} ----> {}'.format(ts, encoder.decode([ts])))

## Изучение данных 



Давайте уделим немного времени, чтобы понять формат данных. Набор данных поставляется предварительно обработанным: каждый пример представляет собой массив целых чисел, представляющих слова обзора фильма.

Текст обзоров был преобразован в целые числа, где каждое целое число представляет определенный фрагмент слова в словаре.

Каждый ярлык представляет собой целочисленное значение 0 или 1, где 0 - отрицательный отзыв, а 1 - положительный отзыв.

Вот как выглядит первый обзор:

In [ ]:
for train_example, train_label in train_data.take(1):
  print('Encoded text:', train_example[:10].numpy())
  print('Label:', train_label.numpy())

`info` структура содержит кодер / декодер. Кодер можно использовать для восстановления исходного текста:

In [ ]:
encoder.decode(train_example)

## Подготовим данные для обучения



Вы захотели создать пакеты обучающих данных для своей модели. Все обзоры имеют разную длину, поэтому используйте [padded_batch](https://www.tensorflow.org/api_docs/python/tf/data/Datase) для выравнивания последовательностей внутри батча (см. [пример](https://www.tensorflow.org/api_docs/python/tf/data/Dataset?hl=it#padded_batch)):

In [ ]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32, tf.compat.v1.data.get_output_shapes(train_data)))

test_batches = (
    test_data
    .padded_batch(32, tf.compat.v1.data.get_output_shapes(test_data)))


In [ ]:
for example_batch, label_batch in train_batches.take(2):
  print("Batch shape:", example_batch.shape)
  print("label shape:", label_batch.shape)

## Построение модели



Нейронная сеть создается слоями стеков - это требует двух основных архитектурных решений:

* Сколько слоев использовать в модели?
* Сколько скрытых юнитов использовать для каждого слоя?
В этом примере входные данные состоят из массива слов-индексов. Предсказываемые метки: 0 или 1. Давайте создадим модель стиля «Непрерывный пакет слов» для этой задачи:

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(encoder.vocab_size, 16),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(1, activation='sigmoid')])

model.summary()

Слои укладываются последовательно для построения классификатора:

1. **Первый слой** - это Embedding-слой. Этот уровень берет словарь, закодированный в целое число, и ищет вектор представления для каждого слова. Эти векторы изучаются как модели обучения. Векторы добавляют измерение к выходному массиву. Полученные размеры: `(batch, sequence, embedding)`. Объяснение параметра embedding см. [здесь](https://qarchive.ru/14323921_sloi__keras_embedding).

   Embedding-слой превращает натуральные числа (индексы) в плотные векторы фиксированного размера. например. [[4], [20]] -> [[0,25, 0,1], [0,6, -0,2]]

  Этот слой можно использовать только как первый слой в модели.

1. Затем [`GlobalAveragePooling1D`](https://peltarion.com/knowledge-center/documentation/modeling-view/build-an-ai-model/blocks/global-average-pooling-1d) уровень возвращает выходной вектор фиксированной длины для каждого примера путем усреднения по измерению последовательности. Это позволяет модели обрабатывать ввод переменной длины самым простым способом.

1. Выходной вектор фиксированной длины проходит через **полносвязанный слой** (Dense-слой) с 16 скрытыми нейронами

1. Последний слой имеет один выходной узел. Используя `sigmoid` функцию активации, это значение является плавающей точкой от 0 до 1, представляющей вероятность или уровень достоверности.


## Функция потерь и оптимизатор



Для модели нам необходимо указать функцию потерь и оптимизатор для обучения. Поскольку мы решаем задачу бинарной классификации и на выходе модели будут вероятности (слой из единственного элемента с сигмоидой в качестве функции активации), то мы воспользуемся функцией потерь `binary_crossentropy`.

Это не единственный выбор для функции потерь: Вы можете, например, выбрать `mean_squared_error`. Но обычно `binary_crossentropy` лучше справляется с вероятностями - она измеряет "дистанцию" между распределениями вероятностей, или, как в нашем случае, между истинным распределением и предсказаниями.

Настроим модель с использованием оптимизатора и функции потерь:

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Тренировка модели



Обучите модель, передавая Datasetобъект функции соответствия модели. Установите количество эпох.

In [ ]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

## Оценка модели



Давайте посмотрим как работает модель. Она будет возвращать два значения. Потери (число, показывающее нашу ошибку, меньшие значения - лучше) и точность (accuracy).

In [ ]:
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

## График точности и потерь с течением времени



In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()